# CustomFactor

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/nonlinear/doc/CustomFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet gtsam-develop


## Overview

The `CustomFactor` class allows users to define custom error functions and Jacobians, and while it can be used in C++, it is particularly useful for use with the python wrapper.

## Custom Error Function

The `CustomFactor` class allows users to define a custom error function. In C++ it is defined as below:

```c++
using JacobianVector = std::vector<Matrix>;
using CustomErrorFunction = std::function<Vector(const CustomFactor &, const Values &, const JacobianVector *)>;
```

The function will be passed a reference to the factor itself so the keys can be accessed, a `Values` reference, and a writeable vector of Jacobians.

## Usage

To use `CustomFactor`, users must:

1. Define the custom error function that models the specific measurement or constraint.
2. Implement the calculation of the Jacobian matrix for the error function.
3. Define a noise model of the appropriate dimension.
3. Add the `CustomFactor` to a factor graph, specifying
    - the noise model
    - the keys of the variables it depends on
    - the error function

Below is a simple example that mimics a `BetweenFactor<Pose2>`.

In [14]:
import numpy as np
from gtsam import CustomFactor, noiseModel, Values, Pose2

measurement = Pose2(2, 2, np.pi / 2)

def error_func(this: CustomFactor, v: Values, H: list[np.ndarray]):
    """
    Error function that mimics a BetweenFactor
    :param this: reference to the current CustomFactor being evaluated
    :param v: Values object
    :param H: list of references to the Jacobian arrays
    :return: the non-linear error
    """
    key0 = this.keys()[0]
    key1 = this.keys()[1]
    gT1, gT2 = v.atPose2(key0), v.atPose2(key1)
    error = measurement.localCoordinates(gT1.between(gT2))

    if H is not None:
        result = gT1.between(gT2)
        H[0] = -result.inverse().AdjointMap()
        H[1] = np.eye(3)
    return error

# we use an isotropic noise model, and keys 66 and 77
noise_model = noiseModel.Isotropic.Sigma(3, 0.1)
custom_factor = CustomFactor(noise_model, [66, 77], error_func)
print(custom_factor)

CustomFactor on 66, 77
isotropic dim=3 sigma=0.1



Typically, you would not actually call methods of a custom factor directly: a nonlinear optimizer will call `linearize` in every nonlinear iteration. But if you wanted to, here is how you would do it:

In [15]:
values = Values()
values.insert(66, Pose2(1, 2, np.pi / 2))
values.insert(77, Pose2(-1, 4, np.pi))

print("error = ", custom_factor.error(values))
print("Linearized JacobianFactor:\n", custom_factor.linearize(values))

error =  0.0
Linearized JacobianFactor:
   A[66] = [
	-6.12323e-16, -10, -20;
	10, -6.12323e-16, -20;
	-0, -0, -10
]
  A[77] = [
	10, 0, 0;
	0, 10, 0;
	0, 0, 10
]
  b = [ -0 -0 -0 ]
  No noise model



Note: there are not a lot of restrictions on the function, but note there is overhead in calling a python function from within a c++ optimization loop. You can mitigate this by having a python function that leverages batching of measurements.

Some more examples of usage in python are given in [test_custom_factor.py](https://github.com/borglab/gtsam/blob/develop/python/gtsam/tests/test_custom_factor.py) and [CustomFactorExample.py](https://github.com/borglab/gtsam/blob/develop/python/gtsam/examples/CustomFactorExample.py).